# CORONAVIRUS 

In [1]:
# Import Libraries 

import os
import gc
import re
import math
import nltk
import folium
import random
import requests
import pycountry
import numpy as np
import scipy as sp
import pandas as pd
import networkx as nx
import seaborn as sns
from tqdm import tqdm
from scipy import stats
import matplotlib.cm as cm
import plotly.express as px
from sklearn import metrics
from pandas import Timestamp 
from textblob import TextBlob
from wordcloud import WordCloud
from nltk.corpus import wordnet
import matplotlib.pyplot as plt
from IPython.display import HTML
from nltk.corpus import stopwords
from sklearn.utils import shuffle
import plotly.graph_objects as go
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from nltk import WordNetLemmatizer
import plotly.figure_factory as ff
from sklearn.decomposition import PCA
from nltk.stem import WordNetLemmatizer
from plotly.subplots import make_subplots
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import warnings
warnings.filterwarnings("ignore")

In [ ]:
tqdm.pandas()


In [5]:
np.random.seed(0)

env: PYTHONHASHSEED=0


In [ ]:
DATA_PATH = "../input/CORD-19-research-challenge/"
CLEAN_DATA_PATH = "../input/cord-19-eda-parse-json-and-generate-clean-csv/"

pmc_df = pd.read_csv(CLEAN_DATA_PATH + "clean_pmc.csv")
biorxiv_df = pd.read_csv(CLEAN_DATA_PATH + "biorxiv_clean.csv")
comm_use_df = pd.read_csv(CLEAN_DATA_PATH + "clean_comm_use.csv")
noncomm_use_df = pd.read_csv(CLEAN_DATA_PATH + "clean_noncomm_use.csv")

papers_df = pd.concat([pmc_df,
                       biorxiv_df,
                       comm_use_df,
                       noncomm_use_df], axis=0).reset_index(drop=True)


In [ ]:
CORONA_FILE = "../input/corona-virus-report/covid_19_clean_complete.csv"

full_table = pd.read_csv(CORONA_FILE, parse_dates=['Date'])

full_table[['Province/State']] = full_table[['Province/State']].fillna('')
full_table['Country/Region'] = full_table['Country/Region'].replace('Mainland China', 'China')
full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']

cases = ['Confirmed', 'Deaths', 'Recovered', 'Active']
full_table[cases] = full_table[cases].fillna(0)
cases = ['Confirmed', 'Deaths', 'Recovered', 'Active']
full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']

# replacing Mainland china with just China
full_table['Country/Region'] = full_table['Country/Region'].replace('Mainland China', 'China')

# filling missing values 
full_table[['Province/State']] = full_table[['Province/State']].fillna('')
full_table[cases] = full_table[cases].fillna(0)

# cases in the ships
ship = full_table[full_table['Province/State'].str.contains('Grand Princess')|full_table['Country/Region'].str.contains('Cruise Ship')]

# china and the row
china = full_table[full_table['Country/Region']=='China']
row = full_table[full_table['Country/Region']!='China']

# latest
full_latest = full_table[full_table['Date'] == max(full_table['Date'])].reset_index()
china_latest = full_latest[full_latest['Country/Region']=='China']
row_latest = full_latest[full_latest['Country/Region']!='China']

# latest condensed
full_latest_grouped = full_latest.groupby('Country/Region')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
china_latest_grouped = china_latest.groupby('Province/State')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
row_latest_grouped = row_latest.groupby('Country/Region')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()

temp = full_table.groupby(['Country/Region', 'Province/State'])['Confirmed', 'Deaths', 'Recovered', 'Active'].max()
# temp.style.background_gradient(cmap='Reds')

temp = full_table.groupby('Date')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
temp = temp[temp['Date']==max(temp['Date'])].reset_index(drop=True)